In [13]:
import os
import glob
import subprocess as sp
import yaml
import shutil
from tqdm import tqdm
import numpy as np
import datetime
import re

# Convert notebooks to html

In [2]:
os.chdir('/Users/ginoprasad/ginoprasad.github.io')

In [3]:
metadata_path = 'metadata.yaml'
with open(metadata_path) as infile:
    metadata = yaml.safe_load(infile)

In [4]:
for project_notebook_path in metadata['Projects'][:]:
    if not os.path.exists(project_notebook_path):
        print(f"REMOVING {project_notebook_path}")
        metadata['Projects'].remove(project_notebook_path)
        with open(metadata_path, 'w') as outfile:
            yaml.dump(metadata, outfile, default_flow_style=False)

In [5]:
temp_path = f'{os.getcwd()}/projects/temp.html'

In [6]:
max_base_filename_length = 50

In [7]:
def get_notebook_metadata(project_notebook_path):
    notebook_metadata_str = sp.run(f"head -n 200 '{project_notebook_path}'", shell=True, capture_output=True).stdout.decode().split('\n')
    notebook_metadata_str = ''.join(notebook_metadata_str[notebook_metadata_str.index('   "source": [')+1:notebook_metadata_str.index('   ]')])
    
    notebook_metadata = {}
    notebook_metadata['title'] = re.search('(?<=# ).*?(?<=\\\\n)', notebook_metadata_str).group(0)[:-2]
    notebook_metadata['authors'] = re.search('(?<=#### ).*?(?<=\\\\n)', notebook_metadata_str).group(0)[:-2]
    notebook_metadata['date'] = re.search('[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]', notebook_metadata_str).group(0)
    
    return notebook_metadata

In [8]:
project_names, project_paths, project_dates = [], [], []
for project_notebook_path in tqdm(metadata['Projects']):
    notebook_metadata = get_notebook_metadata(project_notebook_path)    
    project_base_path = os.path.basename(project_notebook_path)[:-len('.ipynb')]

    while len(project_base_path) > max_base_filename_length:
        project_base_path = ' '.join(project_base_path.split(' ')[:-1])
    if not project_base_path:
        print(f"\n\n\n\n\tWarning: Project '{notebook_metadata['title']}' Name exceeds recommended length\n\n\n\n")
        project_base_path = notebook_metadata['title']
    notebook_metadata['project_path'] = f'{os.getcwd()}/projects/{project_base_path}.html'
    
    assert notebook_metadata['project_path'] not in project_paths
    project_names.append(notebook_metadata['title'])
    project_paths.append(notebook_metadata['project_path'])
    project_dates.append(notebook_metadata['date'])
    
    if os.path.exists(notebook_metadata['project_path']) and os.path.getmtime(notebook_metadata['project_path']) > os.path.getmtime(project_notebook_path):
        continue
    
    print(project_base_path)
    print(f"Converting {project_notebook_path}")
    sp.run(f"jupyter nbconvert --to html '{project_notebook_path}' --output '{temp_path}'", shell=True)
    print(f"Project Name: {notebook_metadata['title']}")

    with open(temp_path) as infile:
        lines = infile.readlines()


    title = ' '.join(map(lambda x: x[0].upper() + x[1:] if x else x, project_base_path.split('_')))
    lines[5] = lines[5][:len('<title>')] + title + lines[5][lines[5].index('</title>'):]
 
    with open(temp_path, 'w') as outfile:
        lines.insert(5, '<link rel="icon" href="../docs/assets/logo.png"><iframe src="../header.html" style="height: 12rem; width: 100%" frameborder="0" scrolling="no"></iframe>\n')
        outfile.write(''.join(lines))
    os.rename(temp_path, notebook_metadata['project_path'])
    
    print('\n')

datetimes = [datetime.datetime.strptime(project_date, '%m/%d/%Y') for project_date in project_dates]
sort_list = lambda ls: [y[1] for y in sorted(enumerate(ls), key=lambda x: datetimes[x[0]], reverse=True)]
project_names, project_paths, project_dates = map(sort_list, (project_names, project_paths, project_dates))

100%|███████████████████████████| 22/22 [00:00<00:00, 31.91it/s]


In [15]:
glob.glob(f'{os.getcwd()}/projects/*')

['/Users/ginoprasad/ginoprasad.github.io/projects/vae_mnist.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/audio_transcription.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/deconvolution.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/Image Segmenter.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/earth_movers_distance.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/Gaussian_Blur.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/autotune_combined.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/image_deblurring.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/Gino_Prasad_CV.pdf',
 '/Users/ginoprasad/ginoprasad.github.io/projects/unet_implementation.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/edge_detector.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/wordle_bot.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/transformer_seq_to_seq.html',
 '/Users/ginoprasad/ginoprasad.github.io/proje

In [9]:
project_paths

['/Users/ginoprasad/ginoprasad.github.io/projects/earth_movers_distance.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/transformer_seq_to_seq.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/VAE_Face_Simulation.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/audio_transcription.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/cropping_image_logo.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/Stable_Diffusion_Pretrained.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/solution_11.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/complex_inner_product.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/Autotune_MUS_15.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/autotune_combined.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/Phase_Vocoder.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/yin_pitch_prediction.html',
 '/Users/ginoprasad/ginoprasad.github.io/projects/play_note.html',
 '/Users/gino

In [9]:
index_html_path = 'index.html'
index_html_lines = open(index_html_path).readlines()

In [10]:
project_template = "\t\t\t<li><div class=link><a href=\"projects/{}\">{}</a></div><div class='date'><img src='docs/assets/calendar_icon.png'><span class=date>{}</span></div></li>\n"

project_list_index_start = ["Cool Projects" in x for x in index_html_lines].index(True) + 2
project_list_index_end = index_html_lines[project_list_index_start:].index('\t\t</ul>\n') + project_list_index_start

new_project_list =  [project_template.format(os.path.basename(html_path), name, date) for name, html_path, date in zip(project_names, project_paths, project_dates)]
index_html_lines = index_html_lines[:project_list_index_start] + new_project_list + index_html_lines[project_list_index_end:]
index_html_lines[project_list_index_start-2] = f"\t\t<h2> Cool Projects ({len(metadata['Projects'])}) </h2>\n"

# Copying CV and Updating Links

In [11]:
assert shutil.copy(metadata['CV'], f"projects/{os.path.basename(metadata['CV'])}")

In [13]:
tag_dict = {tag: metadata[tag] for tag in ['CV', 'LinkedIn', 'GitHub']}
tag_dict['CV'] = f"projects/{os.path.basename(tag_dict['CV'])}"

In [14]:
for i, line in enumerate(index_html_lines):
    for tag in tag_dict:
        prefix = f"<a id='{tag}' href='"
        if line.startswith(prefix):
            print(line.strip())
            new_line = prefix + tag_dict[tag] + line[len(prefix) + line[len(prefix):].index("'"):]
            print(new_line)
            index_html_lines[i] = new_line
    
    if line.startswith(prefix):
        del tag_dict[tag]

# Writing Updated Index File

In [15]:
with open(index_html_path, 'w') as outfile:
    outfile.write(''.join(index_html_lines))

In [16]:
sp.run(f"cd '{os.getcwd()}'; git add .; git commit -m 'Automated Website Update'; git push origin main", shell=True)

[main 6c30353] Automated Website Update
 2 files changed, 23 insertions(+), 23 deletions(-)


To https://github.com/GinoP123/ginoprasad.github.io.git
   a15feb3..6c30353  main -> main


CompletedProcess(args="cd '/Users/ginoprasad/ginoprasad.github.io'; git add .; git commit -m 'Automated Website Update'; git push origin main", returncode=0)

# Updating Python Script

In [18]:
if hasattr(__builtins__,'__IPYTHON__'):
    sp.run(f"jupyter nbconvert --to script 'update_website.ipynb' --output 'update_website'", shell=True)

[NbConvertApp] Converting notebook update_website.ipynb to script
[NbConvertApp] Writing 6007 bytes to update_website.py
